## **Install Dependencies**

In [0]:
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 123.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## **Configure Kaggle Credentials**

In [0]:
import os

os.environ["KAGGLE_USERNAME"] = "parthpatoliya"
os.environ["KAGGLE_KEY"] = "a57d5617ea7447189a54257868d3370c"

print("Kaggle credentials configured!")


Kaggle credentials configured!


## **Create Database Schema**

In [0]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce
""")


DataFrame[]

## **Create Volume for Data Storage**

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")


DataFrame[]

## **Download Dataset from Kaggle**

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors


100%|██████████| 4.29G/4.29G [01:23<00:00, 55.0MB/s]


## **Extract Downloaded Dataset**

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
unzip -o ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            
total 18G
-rwxrwxrwx 1 spark-2c806fa4-32f7-45b0-af54-1e nogroup 8.4G Jan  8 19:51 2019-Nov.csv
-rwxrwxrwx 1 spark-2c806fa4-32f7-45b0-af54-1e nogroup 5.3G Jan  8 19:53 2019-Oct.csv
-rwxrwxrwx 1 spark-2c806fa4-32f7-45b0-af54-1e nogroup 4.3G Jan  8 19:48 ecommerce-behavior-data-from-multi-category-store.zip


## **Clean Up Zip File**

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
rm -f ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

total 14G
-rwxrwxrwx 1 spark-2c806fa4-32f7-45b0-af54-1e nogroup 8.4G Jan  8 19:51 2019-Nov.csv
-rwxrwxrwx 1 spark-2c806fa4-32f7-45b0-af54-1e nogroup 5.3G Jan  8 19:53 2019-Oct.csv


## **Restart Python Environment**

In [0]:
%restart_python


## **Load Oct 2019 Data**

In [0]:
oct_events = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",
    header=True,
    inferSchema=True
)


## **Load Nov 2019 Data**

In [0]:

nov_events = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",
    header=True,
    inferSchema=True
)

## **Quick View & Quality Check of Loaded Data in Volume**

In [0]:
print(f"October 2019 - Total Events: {oct_events.count():,}")
oct_events.printSchema()


October 2019 - Total Events: 42,448,764
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [0]:
oct_events.show(5, truncate=False)


+-------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|event_time         |event_type|product_id|category_id        |category_code                      |brand   |price  |user_id  |user_session                        |
+-------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|2019-10-01 00:00:00|view      |44600062  |2103807459595387724|NULL                               |shiseido|35.79  |541312140|72d76fde-8bb3-4e00-8c23-a032dfed738c|
|2019-10-01 00:00:00|view      |3900821   |2053013552326770905|appliances.environment.water_heater|aqua    |33.2   |554748717|9333dfbd-b87a-4708-9857-6336556b0fcc|
|2019-10-01 00:00:01|view      |17200506  |2053013559792632471|furniture.living_room.sofa         |NULL    |543.1  |519107250|566511c2-e2e3-422b-b695-cf8e6e792ca8|
|2019-10-01 00:0

In [0]:
print(f"November 2019 - Total Events: {nov_events.count():,}")
nov_events.printSchema()

November 2019 - Total Events: 67,501,979
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [0]:
nov_events.show(5, truncate=False)

+-------------------+----------+----------+-------------------+-------------------------+------+------+---------+------------------------------------+
|event_time         |event_type|product_id|category_id        |category_code            |brand |price |user_id  |user_session                        |
+-------------------+----------+----------+-------------------+-------------------------+------+------+---------+------------------------------------+
|2019-11-01 00:00:00|view      |1003461   |2053013555631882655|electronics.smartphone   |xiaomi|489.07|520088904|4d3b30da-a5e4-49df-b1a8-ba5943f1dd33|
|2019-11-01 00:00:00|view      |5000088   |2053013566100866035|appliances.sewing_machine|janome|293.65|530496790|8e5f4f83-366c-4f70-860e-ca7417414283|
|2019-11-01 00:00:01|view      |17302664  |2053013553853497655|NULL                     |creed |28.31 |561587266|755422e7-9040-477b-9bd2-6a6e8fd97387|
|2019-11-01 00:00:01|view      |3601530   |2053013563810775923|appliances.kitchen.washer|lg   